In [1]:
# try getting the Eleuther SAE running via this lib
from latentsae import Sae
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/Users/enjalot/code/latent-sae/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Triton not installed, using eager implementation of SAE decoder.


In [23]:


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
inputs = tokenizer("Hello, world!", return_tensors="pt")

with torch.inference_mode():
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

Loading checkpoint shards: 100%|██████████| 4/4 [00:34<00:00,  8.68s/it]


In [25]:
inputs = ["We the people hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness."]

In [29]:
tokenized_inputs = tokenizer(inputs, return_tensors="pt",truncation=True)

In [30]:
tokenized_inputs.input_ids.shape

torch.Size([1, 48])

In [31]:
with torch.inference_mode():
   outputs = model(**tokenized_inputs, output_hidden_states=True)
   print("HIDDEN STATES", len(outputs.hidden_states))
   llama_embeddings = outputs.hidden_states

HIDDEN STATES 33


In [33]:
llama_embeddings[23].shape

torch.Size([1, 48, 4096])

In [41]:
# sae = Sae.load_from_hub("EleutherAI/sae-llama-3-8b-32x", hookpoint="layers.10")
sae = Sae.load_from_hub("EleutherAI/sae-llama-3.1-8b-64x", "layers.23")

Fetching 2 files: 100%|██████████| 2/2 [03:42<00:00, 111.49s/it]
Dropping extra args {'multi_topk': True}


: 

In [34]:
sae_latents = sae.encode(llama_embeddings[23])

In [35]:
sae_latents.top_acts.shape

torch.Size([1, 48, 32])

In [36]:
sae_latents.top_indices.shape

torch.Size([1, 48, 32])

In [17]:
sae_latents.top_indices[0][1]

tensor([ 34496,  28859,  32150,  59087,  53862,  78388,  13486,   3613, 110039,
        114296,  56223,  71909,  87574, 111785,  10220,  86508,  79681, 103532,
         69833, 107594, 129910, 115369, 126513, 103850,  11678,  46127,  55643,
         10455,  65354,  82677,  30749,  33673, 106180,   4642, 127695,  40376,
          4796, 104244,  52024,  97119,  74896,  10227, 108945,  73192,  98653,
         31088,  47750, 104827,  65571,  78962,  82057, 122440,  15351,  14032,
         21024,  46235, 106380,  64058,  73782,  53153,  94998,  83077, 121363,
         67037,  95699,   2610,  97276,  88087, 109084, 101368,  81898,  80264,
        119419,  40167, 102092,  16318,  91610,  99694,  30839,  84988,  14935,
         23503,  55780,   7549,  47055,  66790,  88861,  27737,  21785, 122859,
         36030, 115979,  49515,  83508,  13175,  41546,  63746,  84089, 101567,
         52995,  58830,  17459,  52256,  75069, 124610, 100739,  34133,  90252,
         37650,  66793, 124689,  82839, 

In [37]:
64*4096

262144

In [38]:
dir(sae)

['T_destination',
 'W_dec',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_

In [39]:
sae.cfg

SaeConfig(expansion_factor=64, normalize_decoder=True, num_latents=0, k=32, signed=False)